# read in, clean up, data
inflation, EquityReturns, Unemployment,
consumer sentiment, home buyers index
get into linear form for dataframe, database

In [3]:
#dependencies
import sqlite3
import pandas as pd
from datetime import datetime, timedelta

In [4]:
rawdatapath='EconomicDataRaw/'

# read in CPI, convert to inflation, setup monthly dataframe¶


In [3]:
CPI_df=pd.read_excel(rawdatapath+'CPI_bls.xlsx',skiprows=10,header=1)
CPI_df

,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,HALF1,HALF2
0,1913,29.400,29.300,29.300,29.400,29.200,29.300,29.600,29.800,29.900,30.100,30.200,30.100,NaN,NaN
1,1914,30.100,29.800,29.700,29.400,29.600,29.800,30.100,30.500,30.600,30.400,30.500,30.400,NaN,NaN
2,1915,30.300,30.100,29.800,30.100,30.200,30.300,30.300,30.300,30.400,30.700,30.900,31.000,NaN,NaN
3,1916,31.300,31.300,31.600,31.900,32.000,32.400,32.400,32.800,33.400,33.800,34.400,34.600,NaN,NaN
4,1917,35.000,35.800,36.000,37.600,38.400,38.800,38.400,39.000,39.700,40.400,40.500,41.000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,2017,727.439,729.727,730.320,732.486,733.110,733.775,733.269,735.466,739.359,738.893,738.912,738.477,731.143,737.396
105,2018,742.499,745.866,747.554,750.524,753.647,754.848,754.898,755.317,756.194,757.532,754.992,752.584,749.156,755.253
106,2019,754.017,757.204,761.477,765.507,767.138,767.291,768.571,768.533,769.136,770.895,770.482,769.781,762.106,769.566
107,2020,772.768,774.886,773.199,768.029,768.044,772.245,776.152,778.599,779.684,780.008,779.531,780.263,771.529,779.040


In [4]:
# create monthly dates to use in dataframe
start=datetime.datetime(2000,1,1)
end=datetime.datetime(2021,1,31)
months=pd.date_range(start,end,freq="BM")

In [5]:
#create dataframe
df_econ=pd.DataFrame({'Date':months})

In [6]:
#turn cpi from year/month table into monthly list
CPI=[]
for date in df_econ['Date']:
    cpival=CPI_df.loc[CPI_df["Year"]==date.year].iloc[0,[date.month]].values[0]  # values[0] turns 1 element array to a mumber
    CPI.append(cpival)
# convert this to an inflation rate
# divide cpi/prior period cpi, multiply by 12
CPInum=CPI[1:]  
CPIden=CPI[:-1]
inflation=[12*((n/d)-1) for n,d in zip(CPInum,CPIden)]
inflation.insert(0,0)  # had to cut an element of of num, den - get this back to proper spacing and time period
df_econ['inflation']=inflation
df_econ.set_index('Date',inplace=True)
df_econ.head(7)

,inflation
Date,
2000-01-31,0.000000
2000-02-29,0.068802
2000-03-31,0.096717
2000-04-28,0.009360
2000-05-31,0.009353
2000-06-30,0.067757
2000-07-31,0.023233


# equity returns (S&P) 

In [7]:
SandP_df=pd.read_csv(rawdatapath+'SandP_Yahoo.csv')
# only column I want to keep is Close; drop everything else
SandP_df.drop(columns=['Open','High','Low','Adj Close','Volume'],inplace=True)
SandP_df.head()

,Date,Close
0,1985-01-01,179.630005
1,1985-02-01,181.179993
2,1985-03-01,180.660004
3,1985-04-01,179.830002
4,1985-05-01,189.550003


In [8]:
SandP_df.dtypes

Date      object
Close    float64
dtype: object

In [9]:
# turn "Date" column into date-time object
dto=[]                      # list to hold date time object
for date in SandP_df['Date']:
    dto.append(datetime.datetime.strptime(date,'%Y-%m-%d'))
SandP_df['DTdate']=dto
SandP_df.set_index('DTdate',inplace=True)
SandP_df.drop(columns=['Date'],inplace=True)
SandP_df.head()

,Close
DTdate,
1985-01-01,179.630005
1985-02-01,181.179993
1985-03-01,180.660004
1985-04-01,179.830002
1985-05-01,189.550003


In [10]:
qwell=pd.concat([df_econ,SandP_df],axis=1,join='inner')
qwell

,inflation,Close


In [11]:
SandP_df

,Close
DTdate,
1985-01-01,179.630005
1985-02-01,181.179993
1985-03-01,180.660004
1985-04-01,179.830002
1985-05-01,189.550003
...,...
2020-11-01,3621.629883
2020-12-01,3756.070068
2021-01-01,3714.239990


In [12]:
qwell=SandP_df.iloc[1]
qwell

Close    181.179993
Name: 1985-02-01 00:00:00, dtype: float64

In [13]:
df_econ.to_csv('df_econ.csv')

# Home Buyers Index
Change format so that there is a column for year, and a column for month.
Fill null data with prior month's info.
ALSO: We need dates to align by last day of month - Subtract by one day.

In [42]:
HomeBuyers_df=pd.read_excel(rawdatapath+'HomeBuyerIndex_fhfa.xls',header=1)
HomeBuyers_df

,Region,Year,Quarter,Not Seasonally-Adjusted\nPurchase-Only Index \n(1991Q1=100),Seasonally-Adjusted Purchase-Only Index \n(1991Q1=100),Not Seasonally-Adjusted\nPurchase-Only Index\n% Change Over \nPrevious Quarter,Seasonally-Adjusted Purchase-Only Index\n% Change Over \nPrevious Quarter,Not Seasonally-Adjusted\nPurchase-Only Index\n% Change Over \nPrevious 4 Quarters,Seasonally-Adjusted Purchase-Only Index\n% Change Over \nPrevious 4 Quarters
0,USA,1991,1,100.00,100.00,NaN,NaN,NaN,NaN
1,USA,1991,2,100.49,99.98,0.0049,-0.0002,NaN,NaN
2,USA,1991,3,100.79,100.18,0.0029,0.0019,NaN,NaN
3,USA,1991,4,101.44,101.05,0.0065,0.0087,NaN,NaN
4,USA,1992,1,102.25,102.27,0.0079,0.0120,0.0225,0.0227
...,...,...,...,...,...,...,...,...,...
115,USA,2019,4,275.72,275.45,0.0041,0.0129,0.0546,0.0536
116,USA,2020,1,280.63,281.18,0.0178,0.0208,0.0621,0.0615
117,USA,2020,2,287.22,284.22,0.0235,0.0108,0.0569,0.0576
118,USA,2020,3,296.66,293.92,0.0329,0.0341,0.0803,0.0808


In [43]:
HomeBuyers_df.isnull().sum()

Region                                                                               0
Year                                                                                 0
Quarter                                                                              0
Not Seasonally-Adjusted\nPurchase-Only Index \n(1991Q1=100)                          0
Seasonally-Adjusted Purchase-Only Index \n(1991Q1=100)                               0
Not Seasonally-Adjusted\nPurchase-Only Index\n% Change Over \nPrevious Quarter       1
Seasonally-Adjusted Purchase-Only Index\n% Change Over \nPrevious Quarter            1
Not Seasonally-Adjusted\nPurchase-Only Index\n% Change Over \nPrevious 4 Quarters    4
Seasonally-Adjusted Purchase-Only Index\n% Change Over \nPrevious 4 Quarters         4
dtype: int64

In [44]:
#Drop calculated columns
HomeBuyers_df.drop(HomeBuyers_df.columns[[4,5,6,7,8]], axis=1, inplace=True)
HomeBuyers_df

,Region,Year,Quarter,Not Seasonally-Adjusted\nPurchase-Only Index \n(1991Q1=100)
0,USA,1991,1,100.00
1,USA,1991,2,100.49
2,USA,1991,3,100.79
3,USA,1991,4,101.44
4,USA,1992,1,102.25
...,...,...,...,...
115,USA,2019,4,275.72
116,USA,2020,1,280.63
117,USA,2020,2,287.22
118,USA,2020,3,296.66


In [55]:
#HomeBuyers_df['quarter2'] = HomeBuyers_df[(HomeBuyers_df["Year"]) & (HomeBuyers_df["Quarter"])]
df2 = HomeBuyers_df.assign(ColumnA = HomeBuyers_df.Year.astype(str) + 'Q' + HomeBuyers_df.Quarter.astype(str))
df2

,Region,Year,Quarter,Not Seasonally-Adjusted\nPurchase-Only Index \n(1991Q1=100),ColumnA
0,USA,1991,1,100.00,1991Q1
1,USA,1991,2,100.49,1991Q2
2,USA,1991,3,100.79,1991Q3
3,USA,1991,4,101.44,1991Q4
4,USA,1992,1,102.25,1992Q1
...,...,...,...,...,...
115,USA,2019,4,275.72,2019Q4
116,USA,2020,1,280.63,2020Q1
117,USA,2020,2,287.22,2020Q2
118,USA,2020,3,296.66,2020Q3


In [47]:
# Build df for each desired timepoint (monthly from 1991-2020)
Dates = []

start_date = pd.to_datetime('01-01-1991')
for j in pd.date_range(start_date, freq='M', periods=360):
  Dates.append(j)

In [48]:
MonthlyHB_df = pd.DataFrame(Dates)
MonthlyHB_df = MonthlyHB_df.set_axis(["date"], axis=1)

MonthlyHB_df

,date
0,1991-01-31
1,1991-02-28
2,1991-03-31
3,1991-04-30
4,1991-05-31
...,...
355,2020-08-31
356,2020-09-30
357,2020-10-31
358,2020-11-30


In [61]:
# Convert the date to match 'ColumnA' for a merge
MonthlyHB_df['ColumnA'] = pd.PeriodIndex(MonthlyHB_df.date, freq='Q').astype(str)
MonthlyHB_df

,date,ColumnA
0,1991-01-31,1991Q1
1,1991-02-28,1991Q1
2,1991-03-31,1991Q1
3,1991-04-30,1991Q2
4,1991-05-31,1991Q2
...,...,...
355,2020-08-31,2020Q3
356,2020-09-30,2020Q3
357,2020-10-31,2020Q4
358,2020-11-30,2020Q4


In [60]:
#Merge dataframes on ColumnA

left_join_df= pd.merge(MonthlyHB_df, df2, on='ColumnA', how='left')
left_join_df 

,date,ColumnA,Region,Year,Quarter,Not Seasonally-Adjusted\nPurchase-Only Index \n(1991Q1=100)
0,1991-01-31,1991Q1,USA,1991,1,100.00
1,1991-02-28,1991Q1,USA,1991,1,100.00
2,1991-03-31,1991Q1,USA,1991,1,100.00
3,1991-04-30,1991Q2,USA,1991,2,100.49
4,1991-05-31,1991Q2,USA,1991,2,100.49
...,...,...,...,...,...,...
355,2020-08-31,2020Q3,USA,2020,3,296.66
356,2020-09-30,2020Q3,USA,2020,3,296.66
357,2020-10-31,2020Q4,USA,2020,4,305.67
358,2020-11-30,2020Q4,USA,2020,4,305.67


# Consumer Sentiment
Change format so that there is a column for year, and a column for month.
Fill null data with prior month's info

In [6]:
ConsumerSentiment_df=pd.read_excel(rawdatapath+'UMCSENT_fred.xls',skiprows=9,header=1)
ConsumerSentiment_df

,observation_date,UMCSENT
0,1952-11-01,86.2
1,1952-12-01,NaN
2,1953-01-01,NaN
3,1953-02-01,90.7
4,1953-03-01,NaN
...,...,...
814,2020-09-01,80.4
815,2020-10-01,81.8
816,2020-11-01,76.9
817,2020-12-01,80.7


In [7]:
today = datetime.today() - timedelta(days=1)
print(today)

2021-03-03 21:23:53.238802


In [8]:
#Subtract one day from the date to get end of month data
ConsumerSentiment_df['End of Month'] = ConsumerSentiment_df.apply(lambda row: row.observation_date - timedelta(days=1), axis = 1) 
ConsumerSentiment_df

In [11]:
#Break date into year and month columns
ConsumerSentiment_df['year'] = pd.DatetimeIndex(ConsumerSentiment_df['End of Month']).year
ConsumerSentiment_df['month'] = pd.DatetimeIndex(ConsumerSentiment_df['End of Month']).month
ConsumerSentiment_df

,observation_date,UMCSENT,End of Month,year,month
0,1952-11-01,86.2,1952-10-31,1952,10
1,1952-12-01,NaN,1952-11-30,1952,11
2,1953-01-01,NaN,1952-12-31,1952,12
3,1953-02-01,90.7,1953-01-31,1953,1
4,1953-03-01,NaN,1953-02-28,1953,2
...,...,...,...,...,...
814,2020-09-01,80.4,2020-08-31,2020,8
815,2020-10-01,81.8,2020-09-30,2020,9
816,2020-11-01,76.9,2020-10-31,2020,10
817,2020-12-01,80.7,2020-11-30,2020,11


In [12]:
# Fix the NaN UMCSENT scores...
ConsumerSentiment_df.isnull().sum()

observation_date      0
UMCSENT             210
End of Month          0
year                  0
month                 0
dtype: int64

In [17]:
ConsumerSentiment_df.count()

observation_date    819
UMCSENT             609
End of Month        819
year                819
month               819
dtype: int64

In [25]:
#Option1: Drop null data
CS_NullsRemoved = ConsumerSentiment_df.dropna()
CS_NullsRemoved

,observation_date,UMCSENT,End of Month,year,month
0,1952-11-01,86.2,1952-10-31,1952,10
3,1953-02-01,90.7,1953-01-31,1953,1
9,1953-08-01,80.8,1953-07-31,1953,7
12,1953-11-01,80.7,1953-10-31,1953,10
15,1954-02-01,82.0,1954-01-31,1954,1
...,...,...,...,...,...
814,2020-09-01,80.4,2020-08-31,2020,8
815,2020-10-01,81.8,2020-09-30,2020,9
816,2020-11-01,76.9,2020-10-31,2020,10
817,2020-12-01,80.7,2020-11-30,2020,11


In [19]:
#Option2: Generate 210 results: Replace nulls with last non-null score
CS_LastScoreFill = ConsumerSentiment_df.fillna(method="ffill")
CS_LastScoreFill

,observation_date,UMCSENT,End of Month,year,month
0,1952-11-01,86.2,1952-10-31,1952,10
1,1952-12-01,86.2,1952-11-30,1952,11
2,1953-01-01,86.2,1952-12-31,1952,12
3,1953-02-01,90.7,1953-01-31,1953,1
4,1953-03-01,90.7,1953-02-28,1953,2
...,...,...,...,...,...
814,2020-09-01,80.4,2020-08-31,2020,8
815,2020-10-01,81.8,2020-09-30,2020,9
816,2020-11-01,76.9,2020-10-31,2020,10
817,2020-12-01,80.7,2020-11-30,2020,11


In [26]:
CS_LastScoreFill = CS_LastScoreFill.drop(['observation_date', 'End of Month'], axis=1)
CS_LastScoreFill

,UMCSENT,year,month
0,86.2,1952,10
1,86.2,1952,11
2,86.2,1952,12
3,90.7,1953,1
4,90.7,1953,2
...,...,...,...
814,80.4,2020,8
815,81.8,2020,9
816,76.9,2020,10
817,80.7,2020,11
